In [1]:
import pandas as pd
import os
import pickle

from model import Transformer # this is the transformer.py file
import torch
from torch import nn
import numpy as np

### Data Processing

In [5]:
# Data preprocessing

from data_processing import process_file

data_path = "data/"

all_data = pd.DataFrame()

for file in os.listdir(data_path):
    if file.endswith(".txt"):
        file_path = os.path.join(data_path, file)
        loc_df = process_file(file_path)

        # print(file, len(loc_df))
        if len(all_data) == 0:
            all_data = loc_df
        else:
            all_data = pd.merge(all_data, loc_df, on = ["year", "month"])

all_data.to_csv("data/processed_data.csv")
all_data


,year,month,temp_anomaly_00N90N,temp_anomaly_90S90N,temp_anomaly_60S60N,temp_anomaly_20S20N,temp_anomaly_90S60S,temp_anomaly_90S20S,temp_anomaly_00N30N,temp_anomaly_20N90N,temp_anomaly_30S00N,temp_anomaly_30N60N,temp_anomaly_60S30S,temp_anomaly_90S00N,temp_anomaly_60N90N
0,1880,1,-0.428327,-0.367828,-0.385516,-0.470565,0.594499,-0.224642,-0.314540,-0.473735,-0.466897,-0.587422,-0.245249,-0.321349,-0.265796
1,1880,2,-0.633566,-0.479081,-0.503620,-0.363749,1.015887,-0.279927,-0.346496,-0.859633,-0.489179,-1.127827,-0.250420,-0.342815,0.034542
2,1880,3,-0.607099,-0.444028,-0.455160,-0.186829,0.409663,-0.302381,-0.090030,-0.879228,-0.410413,-1.308984,-0.228976,-0.302125,-0.378169
3,1880,4,-0.382325,-0.374990,-0.385181,-0.292633,0.721319,-0.319337,0.040845,-0.506497,-0.542937,-0.842017,-0.254012,-0.368574,-0.285724
4,1880,5,-0.305193,-0.415901,-0.425577,-0.477365,0.369175,-0.483292,-0.282362,-0.303430,-0.672377,-0.326057,-0.432202,-0.520792,-0.305153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711,2022,8,0.982607,0.613061,0.569696,0.220776,0.342118,0.247630,0.476373,1.355447,0.193023,1.557818,0.221110,0.208904,1.380191
1712,2022,9,0.955873,0.594641,0.572964,0.243651,0.175700,0.213590,0.486164,1.300414,0.175039,1.572939,0.230871,0.197730,1.035945
1713,2022,10,1.003740,0.632713,0.537141,0.247111,0.527499,0.256682,0.427559,1.380687,0.212318,1.438089,0.226281,0.229366,2.217422
1714,2022,11,0.794545,0.478981,0.463640,0.206005,-0.504358,0.182402,0.530329,1.043458,0.075191,1.066647,0.299289,0.140799,1.115256


In [39]:
# Segmentation

from data_processing import create_segments

input_years = 10  # Length of input period in years
target_years = 1   # Length of target period in years

overlapping = True
input_segments, target_segments = create_segments(all_data, input_years, target_years, overlapping=overlapping)

print("# of segments:", len(input_segments))
print("Length of each input vector (months):", len(input_segments[0]))
print("Length of each target vector a(months):", len(target_segments[0]))

# Convert to tensors
feature_columns = all_data.columns.difference(['year', 'month'], sort=False)
input_tensors = torch.tensor([df[feature_columns].values for df in input_segments], dtype=torch.float32)
target_tensors = torch.tensor([df[feature_columns].values for df in target_segments], dtype=torch.float32)

# Print tensor lengths
print("\nTensor Lengths:")
print(f"There are {len(input_tensors)} input tensors and {len(target_tensors)} target tensors")
print(f"Input tensors length: {len(input_tensors[0])}")
print(f"Target tensors length: {len(target_tensors[0])}")

# Prepare the data to be saved
data_to_save = {
    'input_tensors': input_tensors,
    'target_tensors': target_tensors

}
# Dumping input and target arrays into a pickle file
with open('data/tensor_data.pkl', 'wb') as f:
    pickle.dump(data_to_save, f)

# of segments: 132
Length of each input vector (months): 120
Length of each target vector a(months): 12

Tensor Lengths:
There are 132 input tensors and 132 target tensors
Input tensors length: 120
Target tensors length: 12


In [40]:
from dataset import TimeSeriesDataset
from torch.utils.data import DataLoader, random_split
batch_size = 1

ts_dataset = TimeSeriesDataset(input_tensors, target_tensors)
train_size = int(0.6 * len(ts_dataset))  # e.g., 70% of data for training
val_size = int(0.2 * len(ts_dataset))  # 20% of data for validation 
test_size = len(ts_dataset) - train_size - val_size  # Remaining for testing

# Randomly split the dataset into training and validation datasets
train_dataset, val_dataset, test_dataset = random_split(ts_dataset, [train_size, val_size, test_size])

# Create DataLoaders for both training and validation sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)  # Usually, no need to shuffle the validation set
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Train Loader Length: {len(train_loader)}")
print(f"Validation Loader Length: {len(val_loader)}")
print(f"Test Loader Length: {len(test_loader)}")

input_batch, target_batch = next(iter(train_loader))

# Print the shapes
print(f'Input batch shape: {input_batch.shape}')  # e.g., torch.Size([8, 120, 15])
print(f'Target batch shape: {target_batch.shape}')  # e.g., torch.Size([8, 12, 15])

Train Loader Length: 79
Validation Loader Length: 26
Test Loader Length: 27
Input batch shape: torch.Size([1, 120, 13])
Target batch shape: torch.Size([1, 12, 13])


In [41]:
from sklearn.metrics import r2_score

def testing_metrics(logits_batch, target_batch):
    mse = torch.mean((logits_batch - target_batch) ** 2)
    rmse = torch.sqrt(mse)

    mae = torch.mean(torch.abs(logits_batch - target_batch))

    r_squared_values = []
    for t in range(logits_batch.shape[1]):  # Iterate over time steps
        logits_t = logits_batch[:, t, :].view(-1).detach().numpy()
        target_t = target_batch[:, t, :].view(-1).detach().numpy()
        r_squared_t = r2_score(target_t, logits_t)
        r_squared_values.append(r_squared_t)

    r_squared_avg = sum(r_squared_values) / len(r_squared_values)

    return rmse.item(), mae.item(), r_squared_avg

### Model

In [35]:
# Hyperparameters

d_data = 13
d_model = 15
ffn_hidden = 2048
num_heads = 5
drop_prob = 0.1
num_layers = 1

transformer = Transformer(d_model, d_data, ffn_hidden, num_heads, drop_prob, num_layers)
ar_transformer = Transformer(d_model, d_data, ffn_hidden, num_heads, drop_prob, num_layers)

criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(params=transformer.parameters(), lr = 1e-3)
ar_optimizer = torch.optim.AdamW(params = ar_transformer.parameters(), lr = 1e-3)

### Training Loop

In [36]:
# Multistep Forward Pass

def MultiStepForwardPass(model, input_batch, target_batch):
    predictions = None  # Initialize predictions

    for i in range(len(target_batch[0])):
        if i == 0:
            single_step_input = input_batch
        else:
            single_step_input = torch.cat((input_batch[:, i:, :], predictions), dim=1)

        # Extract the target for the current step
        single_step_target = target_batch[:, i:i+1, :]

        # Forward pass
        single_step_logits = model(single_step_input, single_step_target)

        # Update predictions
        if i == 0:
            predictions = single_step_logits
        else:
            predictions = torch.cat((predictions, single_step_logits), dim=1)

    return predictions

In [37]:
# Custom Training Loop

def train_model(model, epochs, optimizer, criterion, save_freq = 20, auto_reg = False, print_results = True):
    print("Autoregressive Model Training") if auto_reg else print("Single-Shot Model Training")
    for epoch in range(1, epochs+1):
        
        model.train()
        train_loss = 0
        train_rmse = 0
        train_mae = 0
        train_r_squared = 0

        for input_batch, target_batch in train_loader:
            optimizer.zero_grad()

            if auto_reg:
                logits_batch = MultiStepForwardPass(model, input_batch, target_batch)
            else:
                logits_batch = model(input_batch, target_batch)
            
            loss = criterion(logits_batch, target_batch)
            loss.backward()

            optimizer.step()

            train_loss += loss.item()

            rmse, mae, r_squared = testing_metrics(logits_batch, target_batch)
            train_rmse += rmse
            train_mae += mae
            train_r_squared += r_squared

        # Calculate average loss and mean average percent error for the epoch
        train_loss /= len(train_loader)
        train_rmse /= len(train_loader)
        train_mae /= len(train_loader)
        train_r_squared /= len(train_loader)

        model.eval()

        val_loss = 0
        val_rmse = 0
        val_mae = 0
        val_r_squared = 0

        with torch.inference_mode():
            for input_batch, target_batch in val_loader:

                # Forward pass
                if auto_reg:
                    logits_batch = MultiStepForwardPass(model, input_batch, target_batch)
                else:
                    logits_batch = model(input_batch, target_batch)

                # Calculate loss
                loss = criterion(logits_batch, target_batch)
                val_loss += loss.item()

                rmse, mae, r_squared = testing_metrics(logits_batch, target_batch)
                val_rmse += rmse
                val_mae += mae
                val_r_squared += r_squared
            
        val_loss /= len(val_loader)
        val_rmse /= len(val_loader)
        val_mae /= len(val_loader)
        val_r_squared /= len(val_loader)
        
        # Print epoch stats
        if(epoch % save_freq == 0 and print_results):
            print(f'Epoch {epoch}/{epochs} | Train Loss: {train_loss:.4f} | Train RMSE: {train_rmse:.4f} | Train MAE: {train_mae:.4f} | Train R^2: {train_r_squared:.4f}')
            print(f'Validation Loss: {val_loss:.4f} | Validation RMSE: {val_rmse:.4f} | Validation MAE: {val_mae:.4f} | Validation R^2: {val_r_squared:.4f}')
            print("")
    
    # if auto_reg:
    #     torch.save(model.state_dict(), f'saved_models/ar_transformer.pt')
    # else:
    #     torch.save(model.state_dict(), f'saved_models/transformer.pt')

In [38]:
# Custom Testing Loop

def test_model(model, test_loader, criterion, auto_reg = False):
    model.eval()
    test_loss = 0
    test_rmse = 0
    test_mae = 0
    test_r_squared = 0
    all_inputs = []
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for input_batch, target_batch in test_loader:

            if auto_reg:
                logits_batch = MultiStepForwardPass(model, input_batch, target_batch)
            else:
                logits_batch = model(input_batch, target_batch)
            loss = criterion(logits_batch, target_batch)
            test_loss += loss.item()

            rmse, mae, r_squared = testing_metrics(logits_batch, target_batch)
            test_rmse += rmse
            test_mae += mae
            test_r_squared += r_squared

            all_inputs.extend(input_batch.cpu().numpy())
            all_predictions.extend(logits_batch.cpu().numpy())
            all_targets.extend(target_batch.cpu().numpy())

    test_loss /= len(test_loader)
    test_rmse /= len(test_loader)
    test_mae /= len(test_loader)
    test_r_squared /= len(test_loader)

    return test_loss, test_rmse, test_mae, test_r_squared, all_inputs, all_predictions, all_targets


### Training & Testing

In [43]:
# Training the model

train_model(transformer, 100, optimizer, criterion, 20, auto_reg=False, print_results=True)
train_model(ar_transformer, 100, ar_optimizer, criterion, 20, auto_reg=True, print_results=True)

single_shot_results = test_model(transformer, test_loader, criterion)
ar_results = test_model(ar_transformer, test_loader, criterion, auto_reg=True)

if overlapping:
    torch.save(transformer.state_dict(), f'saved_models/transformer.pt')
    torch.save(ar_transformer.state_dict(), f'saved_models/ar_transformer.pt')
else:
    torch.save(transformer.state_dict(), f'saved_models/transformer_no_overlap.pt')
    torch.save(ar_transformer.state_dict(), f'saved_models/ar_transformer_no_overlap.pt')
    

results = {
    'single_shot': single_shot_results,
    'auto_regressive': ar_results
}

if not overlapping:
    with open('results/no_overlap.pkl', 'wb') as f:
        pickle.dump(results, f)

Single-Shot Model Training
Epoch 20/100 | Train Loss: 0.0013 | Train RMSE: 0.0333 | Train MAE: 0.0238 | Train R^2: 0.9824
Validation Loss: 0.0010 | Validation RMSE: 0.0304 | Validation MAE: 0.0226 | Validation R^2: 0.9848

Epoch 40/100 | Train Loss: 0.0016 | Train RMSE: 0.0364 | Train MAE: 0.0269 | Train R^2: 0.9748
Validation Loss: 0.0010 | Validation RMSE: 0.0303 | Validation MAE: 0.0239 | Validation R^2: 0.9831

Epoch 60/100 | Train Loss: 0.0006 | Train RMSE: 0.0222 | Train MAE: 0.0157 | Train R^2: 0.9922
Validation Loss: 0.0005 | Validation RMSE: 0.0203 | Validation MAE: 0.0153 | Validation R^2: 0.9932

Epoch 80/100 | Train Loss: 0.0006 | Train RMSE: 0.0230 | Train MAE: 0.0168 | Train R^2: 0.9894
Validation Loss: 0.0006 | Validation RMSE: 0.0221 | Validation MAE: 0.0162 | Validation R^2: 0.9934

Epoch 100/100 | Train Loss: 0.0005 | Train RMSE: 0.0208 | Train MAE: 0.0155 | Train R^2: 0.9931
Validation Loss: 0.0003 | Validation RMSE: 0.0174 | Validation MAE: 0.0132 | Validation R^2: 

In [44]:
# Data Transfer

# Function to test the model and get metrics
def test_and_get_metrics(model, loader, criterion, auto_reg=False):
    test_loss, test_rmse, test_mae, test_r_squared, inputs, predictions, targets = test_model(model, loader, criterion, auto_reg)
    test_results = {
        'loss': test_loss,
        'rmse': test_rmse,
        'mae': test_mae,
        'r_squared': test_r_squared,
        'inputs': inputs,
        'predictions': predictions,
        'targets': targets
    }
    return test_results

# Testing single-shot model
single_shot_test_results = test_and_get_metrics(transformer, test_loader, criterion)
with open('results/single_shot_test_results.pkl', 'wb') as f:
    pickle.dump(single_shot_test_results, f)

# Testing autoregressive model
ar_test_results = test_and_get_metrics(ar_transformer, test_loader, criterion, auto_reg=True)
with open('results/auto_reg_test_results.pkl', 'wb') as f:
    pickle.dump(ar_test_results, f)

# Testing autoregressive with single-shot model
auto_reg_single_shot_test_results = test_and_get_metrics(transformer, test_loader, criterion, auto_reg=True)
with open('results/auto_reg_single_shot_test_results.pkl', 'wb') as f:
    pickle.dump(auto_reg_single_shot_test_results, f)


### Finding Optimal Model Size

In [28]:
# Optimize Number of Layers

def optimize_model(num_layers_range, epochs, train_loader, val_loader, test_loader, criterion, save_freq=20):
    return
    results = []

    for i in range(1,11):
        print(f"Testing with {i} layers...")

        transformer = Transformer(d_model, ffn_hidden, num_heads, drop_prob, i)
        ar_transformer = Transformer(d_model, ffn_hidden, num_heads, drop_prob, i)

        optimizer = torch.optim.AdamW(params=transformer.parameters(), lr=1e-3)
        ar_optimizer = torch.optim.AdamW(params=ar_transformer.parameters(), lr=1e-3)

        train_model(transformer, epochs, optimizer, criterion, save_freq, auto_reg=False, print_results=False)
        train_model(ar_transformer, epochs, ar_optimizer, criterion, save_freq, auto_reg=True, print_results=False)

        single_shot_results = test_model(transformer, test_loader, criterion)
        ar_results = test_model(ar_transformer, test_loader, criterion, auto_reg=True)

        result = {
            'num_layers': i,
            'single_shot': single_shot_results,
            'auto_regressive': ar_results
        }
        results.append(result)

        torch.save(transformer.state_dict(), f'saved_models/transformer_{i}_layers.pt')
        torch.save(ar_transformer.state_dict(), f'saved_models/ar_transformer_{i}_layers.pt')
    
    with open('optimization_results.pkl', 'wb') as f:
        pickle.dump(results, f)

    return results

num_layers_range = range(1, 11)
epochs = 100  # or whatever number of epochs you deem appropriate
optimization_results = optimize_model(num_layers_range, epochs, train_loader, val_loader, test_loader, criterion)